# Autograd
- 파이토치를 이용해 코드를 작성할 때, Back Propagation 을 이용해 파라미터를 업데이트하는 방법은 Autograd 방식으로 구현 돼 있다.

In [37]:
import torch
print("torch version : {}".format(torch.__version__))
if torch.cuda.is_available():
    Device = 'cuda'
else:
    Device = 'cpu'
print("torch cuda available : {}".format(Device))

torch version : 1.7.1+cu110
torch cuda available : cuda


In [38]:
# 딥러닝에서 파라미터를 업데이트할 때 계산되는 데이터 개수이다.
# 즉, Batch_size 만큼 데이터를 이용해 Output을 계산하고 Batch_size 수만큼 출력된 결과값에 대한 오차값을 계산한다.
Batch_size = 64

# Input_size 는 입력크기이며, 입력층의 노드 수를 말한다.
# Batch_size와 혼동하면 안된다. 1000크기의 벡터값을 64개에 이용한다는 의미 = (64,1000)
Input_size = 1000

# Hidden_size 는 딥러닝 모델에서 Input 을 다수의 파라미터를 이용해 계산한 결과에 한 번 더 계산되는 파라미터 수를 의미한다.
Hidden_size = 100

# Output_size 는 출력값이다.
Output_size = 10

In [39]:
# torch.randn(x,y) 는 x,y크기 만큼 평균이 0, 분산이 1인 표준정규분포에서 데이터를 랜덤하게 뽑아준다.
x = torch.randn(Batch_size, Input_size, device = Device, dtype = torch.float, requires_grad=False)
y = torch.randn(Batch_size, Output_size, device = Device, dtype = torch.float, requires_grad=False)

# 미분값이 필요한 것은 아래 가중치만 필요하기 때문에 required_grad = True로 맞춰주기
w1 = torch.randn(Input_size, Hidden_size, device = Device, dtype = torch.float, requires_grad=True)
w2 = torch.randn(Hidden_size, Output_size, device = Device, dtype = torch.float, requires_grad=True)

In [40]:
learning_rate = 1e-6
# 500번 반복해 결과값을 업데이트할 것
for t in range(1,501):
    # y의 결과값은 x값과 w1을 텐서곱한다. 이 값을 clamp(비선형 함수)를 적용시킨다. 이후 다시 한 번 이 값을 w2와 곱한다.
    # 여기서의 clamp는 Relu와 비슷한 역할을 한다. 0 아래로는 0으로 만들어주고 이후 나머지는 본인의 값을 갖는다.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # 위 식을 통해 예측한 y_pred 값과 y값을 빼고 각 값들을 모두 제곱을 한 후 모두 더한다.
    loss = (y_pred - y).pow(2).sum()

    # t / 100의 나머지가 0이 된다면 몇 번 반복했는지, loss값은 몇 인지 출력 해야한다.
    if t % 100 == 0:
        print('Iteration : ',t,"\t","Loss : ",loss.item())
    
    # loss값에 대해 Backward Propagation 진행
    loss.backward()

    # 아래 코드가 실행되면 Gradient 가 고정된다.
    # torch.no_grad()의 주된 목적은 autograd를 끔으로써 메모리 사용량을 줄이고 연산 속도를 높히기 위함이다. 
    # 사실상 어짜피 안쓸 gradient인데 inference시에 굳이 계산할 필요가 없지 않은가?
    with torch.no_grad():
        # w1,w2에 대해 learning_rate만큼 가중치 업데이트
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # 다음에 다시 loss.backward()연산을 통해 gradient 를 초기화시켜주기 위해, w1 과 w2의 gradient 를 0으로 만들어준다.
        # 하지 않으면, grad값이 자꾸 축적되어 이상한 값이 나온다.
        w1.grad.zero_()
        w2.grad.zero_()

Iteration :  100 	 Loss :  557.702392578125
Iteration :  200 	 Loss :  2.3244500160217285
Iteration :  300 	 Loss :  0.04844530671834946
Iteration :  400 	 Loss :  0.03135206550359726
Iteration :  500 	 Loss :  0.04011261463165283
